In [149]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from tqdm import tqdm
import time


import seaborn as sns
sns.set_style("whitegrid")

import ukcensusapi.Nomisweb as census_api

import urllib3
import json

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
# Get all dataset on nomis: https://www.nomisweb.co.uk/api/v01/dataset/def.htm

In [86]:
# Age and sex by region: NM_792_1
# Age only: NM_503_1

regions = {
    'Camden': 1946157246,
    'City of London': 1946157247,
    'Hackney': 1946157248,
    'Haringey': 1946157250,
    'Islington': 1946157251,
    'Kensington and Chelsea': 1946157252,
    'Lambeth': 1946157253,
    'Lewisham': 1946157254,
    'Newham': 1946157255,
    'Southwark': 1946157256,
    'Tower Hamlets': 1946157257,
    'Wandsworth': 1946157258,
    'Westminster': 1946157259,
    'Barking and Dagenham': 1946157260,
    'Barnet': 1946157261,
    'Bexley': 1946157262,
    'Brent': 1946157263,
    'Bromley': 1946157264,
    'Croydon': 1946157265,
    'Ealing': 1946157266,
    'Enfield': 1946157267,
    'Greenwich': 1946157268,
    'Harrow': 1946157269,
    'Havering': 1946157270,
    'Hillingdon': 1946157271,
    'Hounslow': 1946157272,
    'Kingston upon Thames': 1946157273,
    'Merton': 1946157274,
    'Redbridge': 1946157275,
    'Richmond upon Thames': 1946157276,
    'Sutton': 1946157277,
    'Waltham Forest': 1946157278,
    'Dartford': 1946157313,
    'Elmbridge': 1946157328,
    'Epping Forest': 1946157216,
    'Epsom and Ewell': 1946157329,
    'Hammersmith & Fulham': 1946157249,
    'Hertsmere': 1946157225,
    'Mole Valley': 1946157331,
    'Reigate and Banstead': 1946157332,
    'Runnymede': 1946157333,
    'Sevenoaks': 1946157317,
    'South Bucks': 1946157293,
    'Spelthorne': 1946157334,
    'Tandridge': 1946157336,
    'Three Rivers': 1946157229,
    'Thurrock': 1946157204,
    'Watford': 1946157230,
    'Woking': 1946157338
}


In [66]:
http = urllib3.PoolManager()

In [152]:
for r in tqdm(regions):
    # Deal with age and sex table
    res = http.request("GET", "https://www.nomisweb.co.uk/api/v01/dataset/NM_792_1.data.csv?geography={}&rows=c_age&cols=c_sex".format(regions[r]))

    with open('tmp.txt', 'w') as f:
        f.write(res.data.decode('utf-8'))

    df = pd.read_csv('tmp.txt')
    df = df.sort_values('C_AGE')
    df = df.iloc[1:, :]
    df.index = list(range(len(df)))

    df = df[['GROUP 1: OBS_VALUE', 'GROUP 2: OBS_VALUE', 'GROUP 3: OBS_VALUE']]

    df.rename(columns={'GROUP 1: OBS_VALUE': 'All', 'GROUP 2: OBS_VALUE': 'Males', 'GROUP 3: OBS_VALUE': 'Females'}, inplace=True)

    df.to_csv('../../data/nomis_tmp/{}_age_sex.csv'.format(r), index=False)

    # Deal with age table
    res = http.request("GET", "https://www.nomisweb.co.uk/api/v01/dataset/NM_503_1.data.csv?geography={}&rows=c_age&cols=rural_urban&measures=20100".format(regions[r]))

    with open('tmp.txt', 'w') as f:
        f.write(res.data.decode('utf-8'))

    df = pd.read_csv('tmp.txt')

    df = df.sort_values('C_AGE')
    df = df.iloc[1:, :]
    df.index = list(range(len(df)))

    df = df['GROUP 1: OBS_VALUE']

    df.rename('count', inplace=True)

    df.to_csv('../../data/nomis_tmp/{}_age.csv'.format(r), index=False)

    time.sleep(1)

100%|██████████| 49/49 [01:17<00:00,  1.58s/it]
